In [1]:
from tta_evaluate import evaluate_tta

target_sets = ['PTB-XL', 'Ningbo', 'CPSC+Extra', 'G12EC', 'Chapman']

for target in target_sets:
    evaluate_tta(source='Chapman', target=target)

/home/painstudy/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Source: Chapman, Target: PTB-XL
 ### Leads: 12
Copying data...Done.
Loading configurations...Done.
Loading train set & model...Done
1. Baseline...


100%|██████████| 152/152 [01:36<00:00,  1.57it/s]


Evaluating model...
- AUROC and AUPRC...
auroc 0.763, auprc 0.334
- Accuracy...
acc 0.278
- F-measure...
f score 0.272
- Challenge metric...
challenge metric: 0.056333152312323304
Done.
2. Norm method...


100%|██████████| 152/152 [01:27<00:00,  1.73it/s]


...

Evaluating model...
- AUROC and AUPRC...
auroc 0.763, auprc 0.334
- Accuracy...
acc 0.276
- F-measure...
f score 0.270
- Challenge metric...
challenge metric: 0.05266495103231722
Done.
4. Tent - online method...


100%|██████████| 152/152 [04:10<00:00,  1.65s/it]


Evaluating model...
- AUROC and AUPRC...
auroc 0.765, auprc 0.337
- Accuracy...
acc 0.280
- F-measure...
f score 0.270
- Challenge metric...
challenge metric: 0.05933253945488318
Done.
Source: Chapman, Target: Ningbo
 ### Leads: 12
Copying data...Done.
Loading configurations...Done.
Loading train set & model...Done
1. Baseline...


 90%|████████▉ | 218/243 [02:37<00:18,  1.38it/s]


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "/home/painstudy/.local/lib/python3.8/site-packages/scipy/io/matlab/mio.py", line 39, in _open_file
    return open(file_like, mode), True
FileNotFoundError: [Errno 2] No such file or directory: 'data_aggr/JS27200.mat'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/painstudy/.local/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/painstudy/.local/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/painstudy/.local/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/painstudy/ECG_TTA/DSAIL

In [ ]:
target_sets = ['Ningbo','Chapman', 'CPSC+Extra', 'G12EC', 'PTB-XL'] 
for target in target_sets:
    evaluate_tta(source='PTB-XL', target=target)

## Train baseline model

In [4]:
import torch, os
from team_code import training_code

import data_aggregation
import os, shutil

#os.system("rm -f data_aggr")

source = 'CPSC+Extra'
source_directory = f"./../dataset/{source}/"
goal_directory = "data_train/"
extensions = [".hea", ".mat"]

data_aggregation.copy_files(source_directory, goal_directory, extensions)

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
training_code('data_train', f'model/{source}')

Training for 2leads...
Loading data...


FileNotFoundError: [Errno 2] No such file or directory: 'data_train/JS10651.hea'

In [2]:
import torch, os, shutil
import numpy as np
from tqdm import tqdm

#%load_ext autoreload
#%autoreload 2
import data_aggregation
import src.config as config
from src.data import get_dataset_from_configs, collate_into_list, get_loss_weights_and_flags
from src.model.model_utils import get_model, get_profile
from team_code import load_model
from src.model.model import ECG_model
from tent import tent
import torch.nn.functional as F

# source, target settings
source = 'Ningbo'
target = 'Chapman'

# configuration settings
num_leads = 6

# set target dataset
fdir = 'data_aggr'
if os.path.exists(fdir):
    shutil.rmtree(fdir)
os.mkdir(fdir)

data_aggregation.copy_files(source_dir = f"./../dataset/{target}/", goal_dir = f"{fdir}/", file_extensions = [".hea", ".mat"])

In [ ]:
## initialize a model
model, params = get_model(model_cfg, len(data_cfg.leads), len(data_cfg.scored_classes))
get_profile(model, len(data_cfg.leads), data_cfg.chunk_length)

#model = load_model(model_directory = 'model', leads = '12')[3]
state_dict = torch.load(f'model/{source}/{num_leads}leads_model.pt')
model.load_state_dict(state_dict)

model = tent.configure_model(model)
params, param_names = tent.collect_params(model)
#optimizer = TODO_optimizer(params, lr=1e-3)
optimizer = torch.optim.Adam([{'params': params[0], 'weight_decay': run_cfg.weight_decay},
                              {'params': params[1], 'weight_decay': 0}])
tent_episodic = True
tented_model = tent.Tent(model, optimizer, episodic=tent_episodic)
#outputs = tented_model(dataset_train)

In [4]:
import src.config as config
from src.data import get_dataset_from_configs, collate_into_list, get_loss_weights_and_flags
from src.model.model_utils import get_model, get_profile
from src.train import Trainer
from src.utils import set_seeds
import time, torch

# source, target settings
source = 'Ningbo'
target = 'Chapman'

# configuration settings
num_leads = 6

print('Training for %dleads...' % num_leads)
data_cfg = config.DataConfig(f"config/{target}/cv-{num_leads}leads.json")
preprocess_cfg = config.PreprocessConfig("config/preprocess.json")
model_cfg = config.ModelConfig("config/model.json")
run_cfg = config.RunConfig("config/run.json")
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

print('Loading data...')
set_seeds(2021)
data_directory = fdir
dataset_train = get_dataset_from_configs(data_cfg, preprocess_cfg, data_directory, split_idx = "train")
#data_cfg.filenames = get_filenames_from_split(data_cfg, "all", "val", data_directory)

iterator_train = torch.utils.data.DataLoader(dataset_train, run_cfg.batch_size, #collate_fn=collate_into_list,
                                             shuffle=True, pin_memory=True, num_workers=4)

print("training samples: %d" % len(dataset_train))

## initialize a model
model, params = get_model(model_cfg, len(data_cfg.leads), len(data_cfg.scored_classes))
get_profile(model, len(data_cfg.leads), data_cfg.chunk_length)

# loading model
state_dict = torch.load(f'model/{source}/{num_leads}leads_model.pt')
model.load_state_dict(state_dict)

## setup trainer configurations
loss_weights_and_flags = get_loss_weights_and_flags(data_cfg, dataset_train)
trainer = Trainer(model, data_cfg, loss_weights_and_flags)
trainer.set_device(device)
trainer.set_optim_scheduler(run_cfg, params, len(iterator_train))


for B, batch in enumerate(iterator_train):
    trainer.evaluate(batch, device)

outputs = trainer.logger_test.scalar_outputs[0][0]
outputs_list.append(outputs)


Training for 6leads...
Loading data...


NameError: name 'fdir' is not defined

In [5]:
dataset_train[0][0].shape

NameError: name 'dataset_train' is not defined